# COGS 108 - Data Checkpoint

# Names

- Alex Kuang
- Lucas Papaioannou
- Ryan Harsono
- Bailey Ho
- Michael Maytesyan

<a id='research_question'></a>
# Research Question

How does the change in a state’s public transportation spending affect the sales of different brands of used cars?

# Dataset(s)

**Dataset 1**
- Dataset Name: US Used cars dataset
- Link to the dataset: https://www.kaggle.com/datasets/ananaymital/us-used-cars-dataset
- Number of observations: 3000000
- Description: This dataset contains details of 3 million used cars in the United States. For each car (row), it contains details like body type and fuel economy.

**Dataset 2**
- Dataset Name: FTA Allocations for Formula and Discretionary Programs by State FY 1998-2022 Full Year
- Link to the dataset: https://www.transit.dot.gov/funding/grants/fta-allocations-formula-and-discretionary-programs-state-fy-1998-2022-full-year
- Number of observations: 55
- Description: This dataset contains data of how much was allocated for each state, for each year from 1998 to 2022. Each column contains how much in dollars was allocated for what (ex: Metropolitan Planning, Transit Oriented Development) in each state.

**Dataset 3**
- Dataset Name: US Zipcodes to County State to FIPS Crosswalk
- Link to the dataset: https://www.kaggle.com/datasets/danofer/zipcodes-county-fips-crosswalk
- Number of observations: 52889
- Description: This dataset contains correlations between Zipcodes and the state they belong to. For each row there is a Zipcode, a State value, and other values like county.

We plan to combine Dataset 1 and Dataset 3 as dataset 1 has the zipcode of dealerships but not the states they are in. Since we are interested in states for our research question we would have to merge Dataset 3 into Dataset 1 so it could be aggregated by state. We plan to use Dataset 2 to see if there is a relationship between the % change in spending to the % change in prices for different brands of cars.

# Setup

In [2]:
## YOUR CODE HERE
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
zipdf = pd.read_csv('.\ZIP-COUNTY-FIPS_2017-06.csv')
cardf = pd.read_csv(r'.\used_cars_data.csv', low_memory = True)

C:\Users\omicr\AppData\Local\Temp\ipykernel_86844\1915416024.py:7: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  cardf = pd.read_csv(r'.\used_cars_data.csv', low_memory = True)


# Data Cleaning

To begin, we first checked how much of each of the columns in our dataset had missing data, which is indicated by NaN or NA. We found that the column labeled "franchise_make", representing the different brands of vehicles, was missing data in approximately 19% of its rows, comprising a significant portion of the column data. Because one of our variables of interest in our research question is the brand of vehicles being sold, we dropped rows that have missing data in the "franchise_make" column. 

We then needed to examine if any zip codes in our reduced Used Cars dataset, smaller_cardf, did not appear in the dataset we use to associate US zip codes to each state, represented as zipdf. This was especially important because we are interested in state-specific data in our research goals. Both smaller_cardf and zipdf used a variety of data types to represent zip codes in their dataset such as integers and strings, so we further cleaned the data by standardizing all zip codes to be represented as strings. Additionally, some of the numerically lesser zip codes lacked 0 prefixes, which we addressed by using the zfill method so that each zip code is exactly 5 digits. As a result of both of these methods, we discovered a list of unrecognized zips that were not associated with any US state. We accordingly removed cars from our Used Cars dataset that corresponded with these unrecognized zips, so that our cleaned dataset could effectively track each car sale back to its state location.

In [77]:
# There are many columns currently in this dataset that we do not need. 
# To answer our research question and preform analysis we only need 
# 'dealer_zip', 'price', 'franchise_make', 'body_type', 'is_new', 'year', and 'listed_date'
cardf.head()

,vin,back_legroom,bed,bed_height,bed_length,body_type,cabin,city,city_fuel_economy,combine_fuel_economy,...,transmission,transmission_display,trimId,trim_name,vehicle_damage_category,wheel_system,wheel_system_display,wheelbase,width,year
0,ZACNJABB5KPJ92081,35.1 in,NaN,NaN,NaN,SUV / Crossover,NaN,Bayamon,NaN,NaN,...,A,9-Speed Automatic Overdrive,t83804,Latitude FWD,NaN,FWD,Front-Wheel Drive,101.2 in,79.6 in,2019
1,SALCJ2FX1LH858117,38.1 in,NaN,NaN,NaN,SUV / Crossover,NaN,San Juan,NaN,NaN,...,A,9-Speed Automatic Overdrive,t86759,S AWD,NaN,AWD,All-Wheel Drive,107.9 in,85.6 in,2020
2,JF1VA2M67G9829723,35.4 in,NaN,NaN,NaN,Sedan,NaN,Guaynabo,17.0,NaN,...,M,6-Speed Manual,t58994,Base,NaN,AWD,All-Wheel Drive,104.3 in,78.9 in,2016
3,SALRR2RV0L2433391,37.6 in,NaN,NaN,NaN,SUV / Crossover,NaN,San Juan,NaN,NaN,...,A,8-Speed Automatic Overdrive,t86074,V6 HSE AWD,NaN,AWD,All-Wheel Drive,115 in,87.4 in,2020
4,SALCJ2FXXLH862327,38.1 in,NaN,NaN,NaN,SUV / Crossover,NaN,San Juan,NaN,NaN,...,A,9-Speed Automatic Overdrive,t86759,S AWD,NaN,AWD,All-Wheel Drive,107.9 in,85.6 in,2020


In [78]:
# Since we do not need all the columns of this df we just create another df that contains the columns we need.
needed_cols = ['dealer_zip', 'price', 'franchise_make', 'body_type', 'is_new', 'year','listed_date']
smaller_cardf = cardf[needed_cols]

# checking how much of each column is NaN or NA
print(smaller_cardf.isna().mean())
# We can see that the "franchise_make" column has a sizeable percentage of missing data, around 19%. 
# Since we are interested in "franchise_make" for our research question we have to drop columns that have a missing franchise_make
smaller_cardf = smaller_cardf[smaller_cardf.notna()['franchise_make']]

dealer_zip        0.000000
price             0.000000
franchise_make    0.190876
body_type         0.004514
is_new            0.000000
year              0.000000
listed_date       0.000000
dtype: float64


In [67]:
# We now need to see if any zips in the smaller_cardf are not in zipdf. 
# It is important that we check this as we are interested in the states for our research question
# We need to convert the ZIP column in zipdf to strings so we can compare them the dealer_zip in smaller_cardf.
zipdf["ZIP"] = zipdf["ZIP"].apply(lambda x: str(x))
# We also need to do the same for smaller_cardf as some zips are ints and others are strings.
smaller_cardf["dealer_zip"] = smaller_cardf["dealer_zip"].apply(lambda x: str(x))
# Some of the zipcodes in zipdf and smaller_cardf are also lacking padding 0s in the front when they are less than 5 digits long.
# We need to fix this by using the zfill method.
zipdf["ZIP"] = zipdf["ZIP"].apply(lambda x: x.zfill(5))
smaller_cardf["dealer_zip"] = smaller_cardf["dealer_zip"].apply(lambda x: x.zfill(5))
# Now that the dfs have been standardized we can find the zips that are not in the zipdf dataset.
zips = zipdf["ZIP"].values

car_zips = smaller_cardf['dealer_zip'].unique()

unknown_zips = []

for i in car_zips:
    if i not in zips:
        unknown_zips.append(i)
        
        
# These are the zips that do not have a corrolating state. We need to filter these zips out of smaller_cardf.
print(unknown_zips)
smaller_cardf = smaller_cardf[smaller_cardf['dealer_zip'].isin(unknown_zips).apply(lambda x: not x)]

['96740-1631', '08816-4351', '14221-6032', '54221', '89520']


In [75]:
# Here is our final dataframe, with only the relevant columns that we want to retain. 
# Unnecessary information has been cut out as we are only interested in the types of cars, 
# when they were listed, whether the model year newer than 2 years., the make, the price, 
# and the zip code of the dealership.

smaller_cardf

,dealer_zip,price,franchise_make,body_type,is_new,year,listed_date
0,00960,23141.0,Jeep,SUV / Crossover,True,2019,2019-04-06
1,00922,46500.0,Land Rover,SUV / Crossover,True,2020,2020-02-15
2,00969,46995.0,FIAT,Sedan,False,2016,2017-04-25
3,00922,67430.0,Land Rover,SUV / Crossover,True,2020,2020-02-26
4,00922,48880.0,Land Rover,SUV / Crossover,True,2020,2020-04-25
...,...,...,...,...,...,...,...
3000032,94559,22901.0,Nissan,Van,True,2020,2020-07-12
3000034,95482,40993.0,Ford,Pickup Truck,False,2017,2020-06-15
3000036,94591,36490.0,Chevrolet,SUV / Crossover,True,2020,2020-03-25
3000037,94559,12990.0,Jeep,Sedan,False,2016,2020-06-13
